# PORVE PAZZE CON VARIATIONAL AUTOECONDERS

### IMPORTING LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import yellowbrick
import shap
import vaex
import vaex.ml
import datetime as dt
from scipy import stats
from scipy.stats import randint as sp_randint
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler, LabelEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error,mean_absolute_percentage_error, r2_score, matthews_corrcoef
from sklearn.utils.class_weight import compute_class_weight
import xgboost as xgb
from bayes_opt import BayesianOptimization

ModuleNotFoundError: No module named 'bayes_opt'

### IMPORTING DATASET

In [ ]:
pd.set_option('display.max_columns',None)
df = pd.read_csv('/content/drive/MyDrive/csv/SupplyChainDataset.csv')

### TRASFORMING ORIGINAL DATASET INTO GROUPED DATASET

In [5]:
agg_functions = {
    'Type': 'first',
    'Order Status': 'first',
    'Order City': 'first',
    'Customer City':'first',
    'Order State': 'first',
    'Customer State':'first',
    'Delivery Status': 'first',
    'Shipping Mode':'first',
   'Days for shipping (real)': 'first',
    'Days for shipping (real)' :'first',
    'order date (DateOrders)': 'first',
    'Benefit per order': ['sum', 'median'],
    'Order Item Discount Rate': 'median',
    'Customer Segment': 'first',
    'Customer Id': 'first',
    'Latitude': 'first',
    'Longitude': 'first',
    #'Category Id': lambda x: ','.join(x.unique()),
    'Category Name': lambda x: ','.join(x.unique()),
    # Add the rest of the categorical columns with the lambda function
    'Order Item Total': ['sum','median'],
    'Order Item Quantity': ['sum']
    # Add any additional numerical columns with the appropriate aggregation functions
}

# Flatten the MultiIndex columns
grouped_df = df.groupby('Order Id').agg(agg_functions).reset_index()
grouped_df.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in grouped_df.columns]

In [6]:
import datetime as dt
grouped_df['order date (DateOrders)_first'] = pd.to_datetime(grouped_df['order date (DateOrders)_first'])
def get_month(x):
    return dt.datetime(x.year, x.month, 1)

grouped_df['InvoiceMonth'] = grouped_df['order date (DateOrders)_first'].apply(get_month)
grouping = grouped_df.groupby('Customer Id_first')['InvoiceMonth']
grouped_df['CohortMonth'] = grouping.transform('min')
grouping_days = grouped_df.groupby('Customer Id_first')['order date (DateOrders)_first']
grouped_df['Cohortdays']  = grouping_days.transform('min')
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    date = df[column].dt.date
    hour = df[column].dt.hour
    return year, month, date, hour

Invoice_Year, Invoice_Month, Invoice_Day, _ = get_date_int(grouped_df, 'order date (DateOrders)_first')
Cohort_Year, Cohort_Month , Cohort_day, _ = get_date_int(grouped_df, 'Cohortdays')
Year_Diff = Invoice_Year - Cohort_Year
Month_Diff = Invoice_Month - Cohort_Month
Day_Diff = Invoice_Day - Cohort_day
grouped_df['CohortIndex_months'] = Year_Diff*12 + Month_Diff +1
grouped_df['CohortIndex_days'] = Day_Diff.dt.days

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


# Convert the comma-separated strings to lists
grouped_df['Category Name_<lambda>'] = grouped_df['Category Name_<lambda>'].apply(lambda x: x.split(','))

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the 'Category Name' column
category_name_encoded = mlb.fit_transform(grouped_df['Category Name_<lambda>'])

# Create a new DataFrame with binary features for each unique item in 'Category Name'
category_name_df = pd.DataFrame(category_name_encoded, columns=mlb.classes_)

# Combine the new binary features with the original DataFrame
grouped_df = pd.concat([grouped_df.drop(columns=['Category Name_<lambda>']), category_name_df], axis=1)

grouped_df['order date (DateOrders)_first'] = pd.to_datetime(grouped_df['order date (DateOrders)_first'])
grouped_df['order date (DateOrders)_first'] = pd.to_datetime(grouped_df['order date (DateOrders)_first'], unit='ms')
grouped_df['Order_Month'] = grouped_df['order date (DateOrders)_first'].dt.month
grouped_df['Order_Day'] =grouped_df['order date (DateOrders)_first'].dt.day
grouped_df['Order_Weekday'] = grouped_df['order date (DateOrders)_first'].dt.dayofweek

grouped_df = vaex.from_pandas(df=grouped_df, copy_index=False)
grouped_df = vaex.ml.CycleTransformer(features=['Order_Month'], n=12).fit_transform(grouped_df)
grouped_df = vaex.ml.CycleTransformer(features=['Order_Day'], n=31).fit_transform(grouped_df)
grouped_df = vaex.ml.CycleTransformer(features=['Order_Weekday'], n=7).fit_transform(grouped_df)
grouped_df = grouped_df.to_pandas_df()

In [7]:
class PreprocessDataFrame_regression:
    def __init__(self, df):
        self.df = df
    
    def drop_canceled(self):
        self.df = self.df[self.df['Order Status_first'] != 'CANCELED']
        return self.df
  
  #def cumulative_orders_customer(self):
     # self.df['Customer_Id'] = self.df['Customer_Id'].astype('object')
      #self.df['total_number_of_transactions_so_far'] = self.df.groupby('Customer_Id').cumcount() + 1

    def process_datetetimes(self):
        self.df.order_date_DateOrders = pd.to_datetime(self.df.order_date_DateOrders)
        (self.df.order_date_DateOrders.astype(np.int64) / int(1e6)).astype('int64')
        return self.df

    def augment_with_network_features(self):
        self.df['Source'] = ('LA' + self.df['Latitude_first'].astype('str') + '-' + 'LO' + self.df['Longitude_first'].astype('str'))
        G = nx.from_pandas_edgelist(self.df, source='Source', target='Order City_first')
        self.df['Source_Centrality'] = self.df.Source.map(dict(G.degree))
        return self.df



#grouped_df = PreprocessDataFrame_regression(grouped_df).add_target_variable()
grouped_df = PreprocessDataFrame_regression(grouped_df).drop_canceled()
grouped_df = PreprocessDataFrame_regression(grouped_df).augment_with_network_features()

In [8]:
grouped_df = grouped_df.drop(['InvoiceMonth','CohortMonth','Cohortdays', 'order date (DateOrders)_first', 'Order Id_', 'Delivery Status_first', 'Longitude_first', 'Latitude_first'], axis=1)
grouped_train, grouped_test = train_test_split(grouped_df, test_size=0.33)
grouped_train = vaex.from_pandas(grouped_train)
grouped_test = vaex.from_pandas(grouped_test)

#grouped train and grouped test
grouped_train = vaex.from_pandas(df=grouped_train, copy_index=False)
grouped_test = vaex.from_pandas(df=grouped_test, copy_index=False)
ohe = vaex.ml.OneHotEncoder(features=["Type_first", 
                                      "Customer Segment_first",'Shipping Mode_first'])
grouped_train = ohe.fit_transform(df=grouped_train)
grouped_test = ohe.transform(grouped_test)

mhe = vaex.ml.MultiHotEncoder(features=["Order City_first", "Customer City_first", 'Customer State_first'])
grouped_train = mhe.fit_transform(df=grouped_train)
grouped_test = mhe.transform(grouped_test)

cols_to_scale = ['Days for shipping (real)_first', 'Benefit per order_sum', 'Benefit per order_median',
                 'Order Item Discount Rate_median', 'Customer Id_first', 'Order Item Total_sum',
                 'Order Item Total_median', 'Order Item Quantity_sum', 'CohortIndex_months',
                 'CohortIndex_days', 'Order_Month', 'Order_Day', 'Order_Weekday', 'Order_Month_x',
                 'Order_Month_y', 'Order_Day_x', 'Order_Day_y', 'Order_Weekday_x', 'Order_Weekday_y',
                 'Source_Centrality']

MinMax = vaex.ml.MinMaxScaler(features = cols_to_scale)

grouped_train = MinMax.fit_transform(df=grouped_train)
grouped_test = MinMax.transform(df=grouped_test)

grouped_train = grouped_test.drop(cols_to_scale)
grouped_test = grouped_test.drop(cols_to_scale)

# Assuming 'grouped_data' contains the preprocessed data with the 'Order Status' column
normal_data_train = grouped_train[grouped_train['Order Status_first'] != 'SUSPECTED_FRAUD']
anomalous_data_train = grouped_train[grouped_train['Order Status_first'] == 'SUSPECTED_FRAUD']

normal_data_test = grouped_test[grouped_test['Order Status_first'] != 'SUSPECTED_FRAUD']
anomalous_data_test = grouped_test[grouped_test['Order Status_first'] == 'SUSPECTED_FRAUD']


# Drop the 'Order Status' column
X_train_normal = normal_data_train.drop(columns=['Order Status_first'])
anomalous_data_train = anomalous_data_train.drop(columns=['Order Status_first'])

X_test_normal = normal_data_test.drop(columns=['Order Status_first'])
anomalous_data_test = anomalous_data_test.drop(columns=['Order Status_first'])

X_train_normal = X_train_normal.to_pandas_df()
X_test_normal = X_test_normal.to_pandas_df()

anomalous_data_train = anomalous_data_train.to_pandas_df()
anomalous_data_test = anomalous_data_test.to_pandas_df()

In [9]:
X_train_normal = X_train_normal[X_train_normal.dtypes[X_train_normal.dtypes != 'object'].index.tolist()]
X_test_normal = X_test_normal[X_test_normal.dtypes[X_test_normal.dtypes != 'object'].index.tolist()]

anomalous_data_train =  anomalous_data_train[anomalous_data_train.dtypes[anomalous_data_train.dtypes != 'object'].index.tolist()]
anomalous_data_test =  anomalous_data_train[anomalous_data_train.dtypes[anomalous_data_train.dtypes != 'object'].index.tolist()]

In [ ]:
#NORMAL VAE

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout
from tensorflow.keras.losses import mse, binary_crossentropy, kl_divergence
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler

tf.compat.v1.disable_eager_execution()


input_dim = X_train_normal.shape[1]
input_shape = (input_dim,)

def build_encoder(input_shape, intermediate_dim, latent_dim):
    inputs = Input(shape=input_shape, name='encoder_input')
    x = Dense(intermediate_dim, activation='relu')(inputs)
    x = Dropout(0.2)(x)
    x = Dense(intermediate_dim // 2, activation='relu')(x)
    x = Dropout(0.2)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)
    return inputs, z_mean, z_log_var

def build_decoder(latent_dim, intermediate_dim, output_shape):
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = Dense(intermediate_dim // 2, activation='relu')(latent_inputs)
    x = Dropout(0.2)(x)
    x = Dense(intermediate_dim, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(output_shape, activation='sigmoid')(x)
    return latent_inputs, outputs

inputs, z_mean, z_log_var = build_encoder(input_shape, 256, 64)
latent_inputs, outputs = build_decoder(64, 256, input_dim)

def sample(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sample, output_shape=(64,), name='z')([z_mean, z_log_var])
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

decoder = Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae_model = Model(inputs, outputs, name='vae_mlp')

def vae_loss(x, x_decoded_mean):
    reconstruction_loss = binary_crossentropy(x, x_decoded_mean) * input_dim
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.square(K.exp(z_log_var)), axis=-1)
    total_loss = K.mean(reconstruction_loss + kl_loss)    
    return total_loss

opt = optimizers.Adam(learning_rate=0.0001)
vae_model.compile(optimizer=opt, loss=vae_loss)
vae_model.summary()

# Train the model
history = vae_model.fit(X_train_normal, X_train_normal,
                        shuffle=True,
                        epochs=1000,
                        batch_size=1000)

# Plot training loss
plt.figure()
plt.plot(history.history['loss'])
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [26]:
from sklearn.metrics import f1_score, recall_score, precision_score

normal_test_reconstructions = vae_model.predict(X_test_normal)
anomalous_test_reconstructions = vae_model.predict(anomalous_data_test)

normal_test_mse = np.mean(np.power(X_test_normal - normal_test_reconstructions, 2), axis=1)
anomalous_test_mse = np.mean(np.power(anomalous_data_test - anomalous_test_reconstructions, 2), axis=1)

y_true = np.concatenate([np.zeros(len(normal_test_mse)), np.ones(len(anomalous_test_mse))])
y_mse = np.concatenate([normal_test_mse, anomalous_test_mse])

thresholds = np.linspace(min(y_mse), max(y_mse), 100)
f1_scores = []

for threshold in thresholds:
    y_pred = (y_mse > threshold).astype('int')
    f1_scores.append(f1_score(y_true, y_pred))

optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal threshold: {optimal_threshold}")

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


Optimal threshold: 0.08003369816332014


In [27]:
y_pred = (y_mse > optimal_threshold).astype('int')

print("Variational Autoencoder:")
print("F1-score:", f1_score(y_true, y_pred))
print("Recall:", recall_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))

print(classification_report(y_true, y_pred))

Variational Autoencoder:
F1-score: 0.045056983832494034
Recall: 0.7157894736842105
Precision: 0.023260586987753986
              precision    recall  f1-score   support

         0.0       0.98      0.31      0.47     20773
         1.0       0.02      0.72      0.05       475

    accuracy                           0.32     21248
   macro avg       0.50      0.51      0.26     21248
weighted avg       0.96      0.32      0.46     21248



### VAE AUTOENCODERS TUNING

In [9]:
import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout
from tensorflow.keras.losses import mse, binary_crossentropy, kl_divergence
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from kerastuner import RandomSearch
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()

input_dim = X_train_normal.shape[1]
input_shape = (input_dim,)

def sample(args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon



def build_model(hp):
    intermediate_dim = hp.Int("intermediate_dim", min_value=32, max_value=256, step=32)
    latent_dim = hp.Int("latent_dim", min_value=16, max_value=128, step=16)
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, step=0.1)
    l2_reg = hp.Float("l2_reg", min_value=0.0, max_value=0.01, step=0.001)
    
    # Encoder
    inputs = Input(shape=input_shape, name='encoder_input')
    x = Dense(intermediate_dim, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(inputs)
    x = Dropout(dropout_rate)(x)
    z_mean = Dense(latent_dim, name='z_mean', kernel_regularizer=regularizers.l2(l2_reg))(x)
    z_log_var = Dense(latent_dim, name='z_log_var', kernel_regularizer=regularizers.l2(l2_reg))(x)
    
    
    # Lambda layer for reparameterization trick
    z = Lambda(sample, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    encoder = Model(inputs, z, name='encoder')

    # Decoder
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = Dense(intermediate_dim, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(latent_inputs)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(input_dim, activation='sigmoid', kernel_regularizer=regularizers.l2(l2_reg))(x)
    decoder = Model(latent_inputs, outputs, name='decoder')

    # Full VAE model
    outputs = decoder(encoder(inputs))
    vae_model = Model(inputs, outputs, name='vae_mlp')
    
    def vae_loss(x, x_decoded_mean):
        # compute the average MSE error, then scale it up, ie. simply sum on all axes
        reconstruction_loss = tf.reduce_sum(tf.math.squared_difference(x, x_decoded_mean))
        # compute the KL loss
        kl_loss = - 0.5 * tf.reduce_sum(1 + z_log_var - tf.math.square(z_mean) - tf.math.square(tf.math.exp(z_log_var)), axis=-1)
        # return the average loss over all 
        total_loss = tf.reduce_mean(reconstruction_loss + kl_loss)    
        return total_loss

    # Compile the model
    learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-3, sampling="LOG")
    opt = optimizers.Adam(learning_rate=learning_rate)
    vae_model.compile(optimizer=opt, loss=vae_loss)
    
    
    return vae_model

# Define the Keras Tuner search space
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=2,
    directory='vae_tuning',
    project_name='vae_hyperparameter_tuning'
)

tuner.search_space_summary()

# Perform hyperparameter tuning
tuner.search(X_train_normal, X_train_normal,
             epochs=300,
             batch_size=1000,
             validation_split=0.1)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

INFO:tensorflow:Reloading Tuner from vae_tuning\vae_hyperparameter_tuning\tuner0.json
Search space summary
Default search space size: 5
intermediate_dim (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
latent_dim (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
dropout_rate (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
l2_reg (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.01, 'step': 0.001, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.001, 'step': None, 'sampling': 'log'}
INFO:tensorflow:Oracle triggered exit


`import kerastuner` is deprecated, please use `import keras_tuner`.


Model: "vae_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 112)]             0         
                                                                 
 encoder (Functional)        (None, 48)                46912     
                                                                 
 decoder (Functional)        (None, 112)               36176     
                                                                 
Total params: 83,088
Trainable params: 83,088
Non-trainable params: 0
_________________________________________________________________


In [35]:
print("Best model input shape:", best_model.layers[0].input_shape)

Best model input shape: [(None, 111)]


In [46]:
print("X_test_normal shape:", X_test_normal.shape)
print("anomalous_data_test shape:", anomalous_data_test.shape)
print("Input dimension:", input_dim)

X_test_normal shape: (20768, 111)
anomalous_data_test shape: (480, 111)
Input dimension: 111


In [56]:
best_hyperparameters = tuner.get_best_hyperparameters()[0]
print("Best hyperparameters:")
print(best_hyperparameters)
best_model.summary()

Best hyperparameters:
Model: "vae_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 111)]             0         
                                                                 
 encoder (Functional)        (None, 48)                46688     
                                                                 
 decoder (Functional)        (None, 111)               35951     
                                                                 
Total params: 82,639
Trainable params: 82,639
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Extract the encoder and decoder models from the best_model
encoder = best_model.get_layer('encoder')
decoder = best_model.get_layer('decoder')

# Encode and reconstruct the normal test data
z_test_normal = encoder.predict(X_test_normal)
reconstructed_normal = decoder.predict(z_test_normal)

# Compute the reconstruction error for normal test data
reconstruction_error_normal = np.mean(np.square(X_test_normal - reconstructed_normal), axis=1)

# Encode and reconstruct the anomalous test data
z_test_fraud = encoder.predict(anomalous_data_test)
reconstructed_fraud = decoder.predict(z_test_fraud)

# Compute the reconstruction error for anomalous test data
reconstruction_error_fraud = np.mean(np.square(anomalous_data_test - reconstructed_fraud), axis=1)

InvalidArgumentError: Matrix size-incompatible: In[0]: [32,111], In[1]: [110,224]
	 [[{{node dense/Relu}}]]

# HIERARCHICAL VAE

In [20]:
import keras
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.losses import mse, binary_crossentropy, kl_divergence
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler

#tf.compat.v1.disable_eager_execution()



input_dim = X_train_normal.shape[1]
input_shape = (input_dim,)
intermediate_dim = input_dim // 2


def build_hierarchical_encoder(input_shape, intermediate_dim, latent_dim_1, latent_dim_2):
    inputs = Input(shape=input_shape, name='encoder_input')
    
    # First level
    x = Dense(intermediate_dim, activation='relu')(inputs)
    z_mean_1 = Dense(latent_dim_1, name='z_mean_1')(x)
    z_log_var_1 = Dense(latent_dim_1, name='z_log_var_1')(x)
    z_1 = Lambda(sample, output_shape=(latent_dim_1,), name='z_1_sampling')([z_mean_1, z_log_var_1])
    
    # Second level
    x = Dense(intermediate_dim, activation='relu')(z_1)
    z_mean_2 = Dense(latent_dim_2, name='z_mean_2')(x)
    z_log_var_2 = Dense(latent_dim_2, name='z_log_var_2')(x)
    z_2 = Lambda(sample, output_shape=(latent_dim_2,), name='z_2_sampling')([z_mean_2, z_log_var_2])
    
    return inputs, z_mean_1, z_log_var_1, z_mean_2, z_log_var_2


def build_hierarchical_decoder(latent_dim_1, latent_dim_2, intermediate_dim, output_shape):
    latent_inputs_1 = Input(shape=(latent_dim_1,), name='z_sampling_1')
    latent_inputs_2 = Input(shape=(latent_dim_2,), name='z_sampling_2')
    
    x = Dense(intermediate_dim, activation='relu')(latent_inputs_2)
    z_1_decoded = Dense(latent_dim_1, activation='sigmoid')(x)
    
    x = Dense(intermediate_dim, activation='relu')(latent_inputs_1)
    outputs = Dense(output_shape, activation='sigmoid')(x)
    
    return latent_inputs_1, latent_inputs_2, z_1_decoded, outputs

def sample(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def hierarchical_vae_loss(y_true, y_pred):
    # Reconstruction loss
    reconstruction_loss = mse(inputs, outputs)
    reconstruction_loss *= input_dim

    # KL divergence loss for the first level
    kl_loss_1 = 1 + z_log_var_1 - K.square(z_mean_1) - K.exp(z_log_var_1)
    kl_loss_1 = -0.5 * K.sum(kl_loss_1, axis=-1)

    # KL divergence loss for the second level
    kl_loss_2 = 1 + z_log_var_2 - K.square(z_mean_2) - K.exp(z_log_var_2)
    kl_loss_2 = -0.5 * K.sum(kl_loss_2, axis=-1)

    return K.mean(reconstruction_loss + kl_loss_1 + kl_loss_2)


latent_dim_1 = int(input_dim / 3)
latent_dim_2 = int(latent_dim_1 / 2)

inputs, z_mean_1, z_log_var_1, z_mean_2, z_log_var_2 = build_hierarchical_encoder(input_shape, intermediate_dim, latent_dim_1, latent_dim_2)
z_1 = Lambda(sample, output_shape=(latent_dim_1,), name='z_1')([z_mean_1, z_log_var_1])
z_2 = Lambda(sample, output_shape=(latent_dim_2,), name='z_2')([z_mean_2, z_log_var_2])
encoder = Model(inputs, [z_1, z_2], name='encoder')

latent_inputs_1, latent_inputs_2, z_1_decoded, outputs = build_hierarchical_decoder(latent_dim_1, latent_dim_2, intermediate_dim, input_dim)
decoder = Model([latent_inputs_1, latent_inputs_2], [z_1_decoded, outputs], name='decoder')

z_1_decoded, outputs = decoder([z_1, z_2])

vae_model = Model(inputs, outputs, name='hierarchical_vae')

opt = optimizers.Adam(learning_rate=0.0001)
vae_model.compile(optimizer=opt, loss=hierarchical_vae_loss)
vae_model.summary()

# Finally, we train the model:
results = vae_model.fit(X_train_normal, X_train_normal,
                        shuffle=True,
                        epochs=1000,
                        batch_size=1000,
                        validation_split=0.1)

Model: "hierarchical_vae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 112)]        0           []                               
                                                                                                  
 dense_45 (Dense)               (None, 56)           6328        ['encoder_input[0][0]']          
                                                                                                  
 z_mean_1 (Dense)               (None, 37)           2109        ['dense_45[0][0]']               
                                                                                                  
 z_log_var_1 (Dense)            (None, 37)           2109        ['dense_45[0][0]']               
                                                                                   

ValueError: Variable <tf.Variable 'dense_47/kernel:0' shape=(18, 56) dtype=float32> has `None` for gradient. Please make sure that all of your ops have a gradient defined (i.e. are differentiable). Common ops without gradient: K.argmax, K.round, K.eval.